In [1]:
import pandas as pd
import numpy as np
import math

data_route = '/content/vgames2.csv'
df = pd.read_csv(data_route, index_col=0) 
df.shape #데이터 확인

FileNotFoundError: ignored

In [ ]:
df.isna().sum()
df.dropna(axis=0, inplace=True)
df.isna().sum()
df.shape # 결측치 제거 확인

In [ ]:
# 장르 전체 현황파악
df['Genre'].value_counts()

In [ ]:
# 장르 전체 현황파악
import matplotlib.pyplot as plt
df['Genre'].value_counts().plot(kind='pie')
plt.ylabel('')
plt.title('Game Genre')

In [ ]:
sorted(df['Year'].unique()) # 2자리 이하의 연도 확인 0~16, 86~98
condition1 = (df['Year'] >= 80) & (df['Year'] <= 99)
condition2 = (df['Year'] >= 0) & (df['Year'] <= 30)

need_to_change1 = condition1[condition1].index
need_to_change2 = condition2[condition2].index

for index in need_to_change1:
  df.loc[index, 'Year'] =  df.loc[index, 'Year'] + 1900
for index in need_to_change2:
  df.loc[index, 'Year'] =  df.loc[index, 'Year'] + 2000

df['Year'].sort_values(ascending=True) # 변환 확인

In [ ]:
# 판매단위 K : 1,000, M = 1,000,000
# 판매량에 M 붙은 값 처리
columns = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']
for col in columns:
  condition = (df[col].str.contains('M'))
  need_to_change = condition[condition].index
  for index in need_to_change:
    df.loc[index, col] = df.loc[index, col].replace('M','')

# K 붙은 값의 index 확인 > 나중에 필요
to_M = []
for col in columns:
  condition = df[col].str.contains('K') 
  need_to_change = condition[condition].index
  to_M.append(need_to_change)

to_M[1:3] # 잘 담겼는지 확인

In [ ]:
# 판매량에 K 붙은 값 처리
for col in columns:
  condition = (df[col].str.contains('K'))
  need_to_change = condition[condition].index
  for index in need_to_change:
    df.loc[index, col] = df.loc[index, col].replace('K','')

# dtype 변형
df = df.astype({'NA_Sales' : 'float', 'EU_Sales' : 'float', 'JP_Sales' : 'float', 'Other_Sales' : 'float'})

# test : K처리된 값들 M과 단위 통일
for index in to_M[0]: 
  df.loc[index, 'NA_Sales'] = df.loc[index, 'NA_Sales'] * 0.001
for index in to_M[1]:  
  df.loc[index, 'EU_Sales'] = df.loc[index, 'EU_Sales'] * 0.001
for index in to_M[2]: 
  df.loc[index, 'JP_Sales'] = df.loc[index, 'JP_Sales'] * 0.001
for index in to_M[3]: 
  df.loc[index, 'Other_Sales'] = df.loc[index, 'Other_Sales'] * 0.001

# 단위 맞추기 적용 잘 됐는지 확인
sorted(df['EU_Sales'].unique()[-50:])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 장르&국가별 시장조사
genre_country = df[['Genre', 'NA_Sales','EU_Sales',	'JP_Sales',	'Other_Sales']]
genre_country.head()

In [ ]:
# 가장 큰 시장 확인
# 게임장르를 제쳐두고 우선 모든 장르 대상으로 가장 출고량 많은 시장확인
# 이상점 염두해 두고 일단 참고지표로 활용.
# 1. 북미 2. 유럽 3. 일본 4. 기타
sales_total = genre_country.loc[:, 'NA_Sales':].sum()

total = pd.DataFrame(sales_total)
total.columns = ['total']
total.plot(kind='pie', y='total', startangle=90, counterclock=False)

plt.legend(loc=(1,1))
plt.title('Sales Total')
plt.ylabel('')
plt.show()

In [ ]:
from matplotlib import rcParams
check1 = genre_country.groupby('Genre').sum().sort_values(by='EU_Sales')
# 정렬기준 column을 바꾸면서 그래프를 출력
check1.reset_index(inplace=True)
check1
# 장르별로 각 지역의 총 판매량 확인(장르별 선호도 확인)

In [ ]:
check1.corr()
# 지역별 상관관계 확인 북미, 유럽, 기타지역의 강한 양의 상관관계 발견

In [ ]:
# 북미지역
rcParams['figure.figsize'] = 11,5
sns.barplot(x='Genre', y='NA_Sales', data=check1)
plt.title('North America', fontsize=18)

In [ ]:
# 유럽지역
sns.barplot(x='Genre', y='EU_Sales', data=check1)
plt.title('Europe', fontsize=18)

In [ ]:
# 일본지역
sns.barplot(x='Genre', y='JP_Sales', data=check1)
plt.title('Japan', fontsize=18)

In [ ]:
# 기타지역
sns.barplot(x='Genre', y='Other_Sales', data=check1)
plt.title('Other country', fontsize=18)

In [ ]:
df.head()

In [ ]:
check2 = df.loc[:, ['Year', 'Genre']]
check2.head()

In [ ]:
# 연도별 게임 제작 장르 수
check2 = check2.groupby(['Year','Genre']).Genre.count()
check2

In [ ]:
# 그래프로 표현하기 쉽게하려면? tidy data
check2_df = pd.DataFrame(check2)
check2_df.rename({'Genre':'Count'}, axis=1, inplace=True)
check2_df.reset_index(inplace=True)
check2_df

In [ ]:
# 연도별 게임 제작 수(장르별) 
sns.lineplot(data=check2_df, x='Year', y='Count', hue='Genre')

In [ ]:
years = check2_df['Year'].unique()

# 연도별 장르 경쟁사 수 확인2
check_years = years[-7:-2]
rcParams['figure.figsize'] = 8,6
for year in check_years:
  condition = (check2_df['Year'] == year)
  print(check2_df[condition].sort_values(by='Count').plot(kind='barh', x='Genre',
  y='Count',colormap='coolwarm', title='{}'.format(int(year))))
  plt.show()

In [ ]:
# 장르별 연도 판매량 확인 = 장르의 연도별 판매 트렌드 파악
temp = df[['Year','Genre','NA_Sales','EU_Sales','JP_Sales','Other_Sales']]
check3 = temp.groupby(['Genre','Year']).agg({'NA_Sales':['sum'], 'EU_Sales':['sum'],'JP_Sales':['sum'], 'Other_Sales':['sum']})
check3

In [ ]:
check3.loc['Action'].plot(linestyle='dashed', title='Action', colormap='coolwarm')
plt.legend()

In [ ]:
check3.loc['Role-Playing'].plot(linestyle='dashed', title='Role-Playing', colormap='coolwarm')
plt.legend()

In [ ]:
check3.loc['Sports'].plot(linestyle='dashed', title='Sports', colormap='coolwarm')
plt.legend()

In [ ]:
check3.loc['Adventure'].plot(linestyle='dashed', title='Adventure', colormap='coolwarm')
plt.legend()

In [ ]:
check3.loc['Shooter'].plot(linestyle='dashed', title='Shooter', colormap='coolwarm')
plt.legend()

In [ ]:
# 게임 플랫폼 시장순위 확인 
df['Platform'].value_counts().sort_values().plot(kind='barh')
# 1. 닌텐도 DS 2. PS 시리즈 3. Wii 4. X360 5.PSP

In [ ]:
# 2012~2016 제작수 1위 플랫폼확인
# 다음 결과를 바탕으로 PS 시리즈 플랫폼으로 선정한 게임장르 개발 진행
df.head(3)
platform = df.groupby(['Year','Platform']).agg({'NA_Sales':['count'], 'EU_Sales':['count'], 'JP_Sales':['count'],'Other_Sales':['count']})
platform.loc[2012.0].sort_values(by=('NA_Sales', 'count'), ascending=False).head(1)

In [ ]:
platform.loc[2013.0].sort_values(by=('NA_Sales', 'count'), ascending=False).head(1)

In [ ]:
platform.loc[2014.0].sort_values(by=('NA_Sales', 'count'), ascending=False).head(1)

In [ ]:
platform.loc[2015.0].sort_values(by=('NA_Sales', 'count'), ascending=False).head(1)

In [ ]:
platform.loc[2016.0].sort_values(by=('NA_Sales', 'count'), ascending=False).head(1)